In [2]:
import tkinter as tk
from math import sin, cos, radians
import random

class AirplaneSimulator:
    def __init__(self, root):
        self.root = root
        self.root.title("Airplane Flight Simulator")
        self.root.geometry("800x600")
        self.root.configure(bg='#1a1a2e')
        
        # Flight parameters
        self.altitude = 5000  # feet
        self.speed = 250  # knots
        self.pitch = 0  # degrees (-90 to 90)
        self.roll = 0  # degrees (-180 to 180)
        self.heading = 0  # degrees (0-360)
        self.throttle = 50  # percentage
        self.vertical_speed = 0  # feet per minute
        
        # Physics
        self.gravity = 0.5
        self.running = True
        
        # Create UI
        self.create_widgets()
        
        # Start simulation
        self.update_simulation()
        
    def create_widgets(self):
        # Main canvas for artificial horizon
        self.canvas = tk.Canvas(self.root, width=600, height=400, bg='#0f3460', highlightthickness=0)
        self.canvas.pack(pady=10)
        
        # Instrument panel
        panel = tk.Frame(self.root, bg='#1a1a2e')
        panel.pack(fill=tk.X, padx=20)
        
        # Create instruments
        self.create_instrument(panel, "Altitude", "altitude_label", 0)
        self.create_instrument(panel, "Speed", "speed_label", 1)
        self.create_instrument(panel, "V/S", "vs_label", 2)
        self.create_instrument(panel, "Heading", "heading_label", 3)
        
        # Control panel
        controls = tk.Frame(self.root, bg='#1a1a2e')
        controls.pack(pady=10)
        
        tk.Label(controls, text="Throttle:", fg='white', bg='#1a1a2e').grid(row=0, column=0, padx=5)
        self.throttle_scale = tk.Scale(controls, from_=0, to=100, orient=tk.HORIZONTAL, 
                                       command=self.update_throttle, bg='#16213e', fg='white',
                                       highlightthickness=0, length=200)
        self.throttle_scale.set(50)
        self.throttle_scale.grid(row=0, column=1, padx=5)
        
        # Instructions
        instructions = tk.Label(self.root, 
                               text="Controls: ↑↓ = Pitch | ←→ = Roll | Q/E = Heading | Throttle Slider",
                               fg='#e94560', bg='#1a1a2e', font=('Arial', 10))
        instructions.pack()
        
        # Bind keys
        self.root.bind('<Up>', lambda e: self.adjust_pitch(2))
        self.root.bind('<Down>', lambda e: self.adjust_pitch(-2))
        self.root.bind('<Left>', lambda e: self.adjust_roll(-3))
        self.root.bind('<Right>', lambda e: self.adjust_roll(3))
        self.root.bind('q', lambda e: self.adjust_heading(-2))
        self.root.bind('e', lambda e: self.adjust_heading(2))
        
    def create_instrument(self, parent, label, attr_name, col):
        frame = tk.Frame(parent, bg='#16213e', relief=tk.RIDGE, borderwidth=2)
        frame.grid(row=0, column=col, padx=10, pady=5)
        
        tk.Label(frame, text=label, fg='#e94560', bg='#16213e', 
                font=('Arial', 10, 'bold')).pack()
        label_widget = tk.Label(frame, text="0", fg='white', bg='#16213e', 
                               font=('Arial', 16), width=8)
        label_widget.pack(pady=5)
        setattr(self, attr_name, label_widget)
        
    def draw_horizon(self):
        self.canvas.delete('all')
        w, h = 600, 400
        cx, cy = w // 2, h // 2
        
        # Calculate horizon position based on pitch
        pitch_offset = self.pitch * 3
        
        # Sky gradient
        sky_y = cy - pitch_offset
        self.canvas.create_rectangle(0, 0, w, sky_y, fill='#4a9eff', outline='')
        
        # Ground
        self.canvas.create_rectangle(0, sky_y, w, h, fill='#8b4513', outline='')
        
        # Horizon line (with roll)
        angle = radians(self.roll)
        length = 800
        x1 = cx - length * cos(angle)
        y1 = sky_y - length * sin(angle)
        x2 = cx + length * cos(angle)
        y2 = sky_y + length * sin(angle)
        self.canvas.create_line(x1, y1, x2, y2, fill='white', width=3)
        
        # Pitch ladder
        for i in range(-30, 40, 10):
            if i == 0:
                continue
            y_pos = cy - pitch_offset - i * 3
            ladder_len = 40 if i % 20 == 0 else 25
            
            # Rotate ladder lines with roll
            dx = ladder_len * cos(angle)
            dy = ladder_len * sin(angle)
            
            self.canvas.create_line(cx - dx, y_pos - dy, cx + dx, y_pos + dy, 
                                   fill='white', width=2)
            self.canvas.create_text(cx + dx + 20, y_pos + dy, text=f"{i}°", 
                                   fill='white', font=('Arial', 8))
        
        # Aircraft symbol (fixed in center)
        wing_size = 60
        self.canvas.create_line(cx - wing_size, cy, cx - 15, cy, fill='yellow', width=4)
        self.canvas.create_line(cx + 15, cy, cx + wing_size, cy, fill='yellow', width=4)
        self.canvas.create_oval(cx - 8, cy - 8, cx + 8, cy + 8, fill='yellow', outline='')
        
        # Roll indicator
        self.canvas.create_arc(cx - 80, 30, cx + 80, 190, start=0, extent=180, 
                              outline='white', width=2, style=tk.ARC)
        # Roll pointer
        pointer_x = cx + 70 * sin(angle)
        pointer_y = 110 - 70 * cos(angle)
        self.canvas.create_polygon(pointer_x, pointer_y,
                                   pointer_x - 5, pointer_y + 10,
                                   pointer_x + 5, pointer_y + 10,
                                   fill='yellow')
        
    def update_throttle(self, val):
        self.throttle = int(val)
        
    def adjust_pitch(self, delta):
        self.pitch = max(-30, min(30, self.pitch + delta))
        
    def adjust_roll(self, delta):
        self.roll = (self.roll + delta) % 360
        if self.roll > 180:
            self.roll -= 360
            
    def adjust_heading(self, delta):
        self.heading = (self.heading + delta) % 360
        
    def update_physics(self):
        # Speed changes based on throttle and pitch
        target_speed = self.throttle * 3 + 50
        speed_diff = target_speed - self.speed
        self.speed += speed_diff * 0.02
        
        # Vertical speed based on pitch and speed
        pitch_effect = self.pitch * 50
        self.vertical_speed = pitch_effect * (self.speed / 250)
        
        # Apply gravity effect
        if self.pitch < 0:
            self.vertical_speed -= self.gravity * 10
            
        # Update altitude
        self.altitude += self.vertical_speed / 60  # Convert to feet per frame
        
        # Ground collision
        if self.altitude < 0:
            self.altitude = 0
            self.vertical_speed = 0
            
        # Roll returns to level gradually
        if abs(self.roll) > 1:
            self.roll *= 0.98
            
    def update_instruments(self):
        self.altitude_label.config(text=f"{int(self.altitude)} ft")
        self.speed_label.config(text=f"{int(self.speed)} kts")
        self.vs_label.config(text=f"{int(self.vertical_speed):+d} fpm")
        self.heading_label.config(text=f"{int(self.heading)}°")
        
    def update_simulation(self):
        if self.running:
            self.update_physics()
            self.draw_horizon()
            self.update_instruments()
            self.root.after(33, self.update_simulation)  # ~30 FPS
            
if __name__ == "__main__":
    root = tk.Tk()
    sim = AirplaneSimulator(root)
    root.mainloop()

In [3]:
import pygame
import sys
import random

# Initialize pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
PADDLE_WIDTH, PADDLE_HEIGHT = 15, 100
BALL_SIZE = 15
FPS = 60
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 120, 255)
RED = (255, 50, 50)
GREEN = (50, 255, 100)
FONT_SIZE = 36

# Create the game window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Professional Ping Pong")
clock = pygame.time.Clock()

# Game objects
player_paddle = pygame.Rect(50, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
opponent_paddle = pygame.Rect(WIDTH - 50 - PADDLE_WIDTH, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)

# Game variables
player_speed = 0
opponent_speed = 7
ball_speed_x = 7 * random.choice((1, -1))
ball_speed_y = 7 * random.choice((1, -1))
player_score = 0
opponent_score = 0
game_font = pygame.font.Font(None, FONT_SIZE)

def reset_ball():
    """Reset the ball to the center with a random direction"""
    ball.center = (WIDTH // 2, HEIGHT // 2)
    return 7 * random.choice((1, -1)), 7 * random.choice((1, -1))

def draw_objects():
    """Draw all game objects on the screen"""
    # Fill background
    screen.fill(BLACK)
    
    # Draw center line
    pygame.draw.aaline(screen, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))
    
    # Draw paddles and ball
    pygame.draw.rect(screen, BLUE, player_paddle)
    pygame.draw.rect(screen, RED, opponent_paddle)
    pygame.draw.ellipse(screen, GREEN, ball)
    
    # Draw scores
    player_text = game_font.render(f"{player_score}", False, WHITE)
    opponent_text = game_font.render(f"{opponent_score}", False, WHITE)
    screen.blit(player_text, (WIDTH // 4, 20))
    screen.blit(opponent_text, (3 * WIDTH // 4 - opponent_text.get_width(), 20))

def move_paddles():
    """Move player and opponent paddles"""
    # Player paddle movement
    player_paddle.y += player_speed
    
    # Keep player paddle on screen
    if player_paddle.top <= 0:
        player_paddle.top = 0
    if player_paddle.bottom >= HEIGHT:
        player_paddle.bottom = HEIGHT
    
    # Opponent AI - follow the ball
    if opponent_paddle.centery < ball.centery:
        opponent_paddle.y += opponent_speed
    elif opponent_paddle.centery > ball.centery:
        opponent_paddle.y -= opponent_speed
    
    # Keep opponent paddle on screen
    if opponent_paddle.top <= 0:
        opponent_paddle.top = 0
    if opponent_paddle.bottom >= HEIGHT:
        opponent_paddle.bottom = HEIGHT

def move_ball():
    """Move the ball and handle collisions"""
    global ball_speed_x, ball_speed_y, player_score, opponent_score
    
    # Move the ball
    ball.x += ball_speed_x
    ball.y += ball_speed_y
    
    # Ball collision with top and bottom
    if ball.top <= 0 or ball.bottom >= HEIGHT:
        ball_speed_y *= -1
        # Add a slight angle variation on wall bounce
        ball_speed_y += random.uniform(-0.5, 0.5)
    
    # Ball collision with paddles
    if ball.colliderect(player_paddle) or ball.colliderect(opponent_paddle):
        ball_speed_x *= -1
        # Add a slight angle variation based on where the ball hits the paddle
        if ball.colliderect(player_paddle):
            ball_speed_x += 0.5  # Increase speed slightly
            # Adjust angle based on where ball hits paddle
            relative_intersect_y = (player_paddle.centery - ball.centery) / (PADDLE_HEIGHT / 2)
            ball_speed_y = -relative_intersect_y * 5
        else:
            ball_speed_x -= 0.5  # Increase speed slightly
            # Adjust angle based on where ball hits paddle
            relative_intersect_y = (opponent_paddle.centery - ball.centery) / (PADDLE_HEIGHT / 2)
            ball_speed_y = -relative_intersect_y * 5
        
        # Limit maximum speed
        ball_speed_x = max(min(ball_speed_x, 12), -12)
        ball_speed_y = max(min(ball_speed_y, 12), -12)
    
    # Ball goes out of bounds
    if ball.left <= 0:
        opponent_score += 1
        ball_speed_x, ball_speed_y = reset_ball()
    if ball.right >= WIDTH:
        player_score += 1
        ball_speed_x, ball_speed_y = reset_ball()

# Main game loop
while True:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        
        # Player paddle controls
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                player_speed = -7
            if event.key == pygame.K_DOWN:
                player_speed = 7
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                player_speed = 0
    
    # Game logic
    move_paddles()
    move_ball()
    
    # Drawing
    draw_objects()
    
    # Update display
    pygame.display.flip()
    clock.tick(FPS)

ModuleNotFoundError: No module named 'pygame'

In [ ]:
import asyncio
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters import CommandStart
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton, CallbackQuery
import asyncpg
import random

# Simple in-memory storage for active users by language
users = {}  # {user_id: partner_id}
queues = {
    'uz': [],  # O'zbek tilida kutayotgan foydalanuvchilar
    'ru': [],  # Rus tilida kutayotgan foydalanuvchilar  
    'en': []   # Ingliz tilida kutayotgan foydalanuvchilar
}
user_languages = {}  # {user_id: 'uz'/'ru'/'en'}

bot = Bot(os.getenv("BOT_TOKEN"))
dp = Dispatcher()

# Translations
texts = {
    'uz': {
        'welcome': "🌍 *Anonim Chatga xush kelibsiz!*\n\nDunyo bo'ylab odamlar bilan anonim suhbatlashing.\nTilni tanlang:",
        'language_selected': "✅ Til tanlandi! Endi xabar yuboring va suhbatni boshlang!\n\nKomandalar:\n/next - Yangi sherik topish\n/stop - Suhbatni to'xtatish\n/language - Tilni o'zgartirish",
        'searching': "🔍 Sherik qidirilmoqda...",
        'connected': "💚 Ulandi! Anonim suhbatni boshlang!",
        'partner_left': "💔 Sherik ketdi. Yangi sherik topish uchun xabar yuboring!",
        'chat_stopped': "❌ Suhbat to'xtatildi. Yangi sherik topish uchun xabar yuboring!",
        'not_in_chat': "❌ Siz hozir suhbatda emassiz. Boshlash uchun xabar yuboring!",
        'select_language': "Tilni tanlang:"
    },
    'ru': {
        'welcome': "🌍 *Добро пожаловать в Анонимный Чат!*\n\nОбщайтесь с людьми по всему миру анонимно.\nВыберите язык:",
        'language_selected': "✅ Язык выбран! Теперь отправьте сообщение и начните общение!\n\nКоманды:\n/next - Найти нового собеседника\n/stop - Остановить чат\n/language - Изменить язык",
        'searching': "🔍 Поиск собеседника...",
        'connected': "💚 Подключен! Начните анонимное общение!",
        'partner_left': "💔 Собеседник ушел. Отправьте сообщение, чтобы найти нового!",
        'chat_stopped': "❌ Чат остановлен. Отправьте сообщение, чтобы найти нового собеседника!",
        'not_in_chat': "❌ Вы не в чате. Отправьте сообщение, чтобы начать!",
        'select_language': "Выберите язык:"
    },
    'en': {
        'welcome': "🌍 *Welcome to Anonymous Chat!*\n\nConnect with random people worldwide anonymously.\nSelect your language:",
        'language_selected': "✅ Language selected! Now send any message to start chatting!\n\nCommands:\n/next - Find new partner\n/stop - Stop current chat\n/language - Change language",
        'searching': "🔍 Searching for partner...",
        'connected': "💚 Connected! Start chatting anonymously!",
        'partner_left': "💔 Partner left. Send any message to find new partner!",
        'chat_stopped': "❌ Chat stopped. Send any message to find new partner!",
        'not_in_chat': "❌ You're not in any chat. Send any message to start!",
        'select_language': "Select language:"
    }
}

def get_text(user_id, key):
    lang = user_languages.get(user_id, 'en')
    return texts[lang][key]

def get_language_keyboard():
    keyboard = InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="🇺🇿 O'zbek", callback_data="lang_uz"),
            InlineKeyboardButton(text="🇷🇺 Русский", callback_data="lang_ru")
        ],
        [
            InlineKeyboardButton(text="🇺🇸 English", callback_data="lang_en")
        ]
    ])
    return keyboard

# Database connection
async def get_db():
    return await asyncpg.connect(os.getenv("DATABASE_URL"))

@dp.message(CommandStart())
async def start(message: types.Message):
    user_id = message.from_user.id
    
    # Check if user already exists in database
    try:
        conn = await get_db()
        user_record = await conn.fetchrow("SELECT * FROM users WHERE user_id = $1", user_id)
        
        if user_record:
            # User exists, load their language
            user_languages[user_id] = user_record['language']
            await message.answer(
                get_text(user_id, 'language_selected'),
                parse_mode="Markdown"
            )
        else:
            # New user, show language selection
            await message.answer(
                texts['en']['welcome'],  # Default to English for welcome
                reply_markup=get_language_keyboard(),
                parse_mode="Markdown"
            )
        
        await conn.close()
    except Exception as e:
        print(f"Database error: {e}")
        # Fallback without database
        await message.answer(
            texts['en']['welcome'],
            reply_markup=get_language_keyboard(),
            parse_mode="Markdown"
        )

@dp.callback_query(lambda c: c.data.startswith('lang_'))
async def language_callback(callback: CallbackQuery):
    user_id = callback.from_user.id
    language = callback.data.split('_')[1]  # uz, ru, en
    
    # Save user language
    user_languages[user_id] = language
    
    # Save to database
    try:
        conn = await get_db()
        await conn.execute("""
            INSERT INTO users (user_id, language, username, first_name) 
            VALUES ($1, $2, $3, $4) 
            ON CONFLICT (user_id) 
            DO UPDATE SET language = $2
        """, user_id, language, callback.from_user.username, callback.from_user.first_name)
        await conn.close()
    except Exception as e:
        print(f"Database error: {e}")
    
    await callback.message.edit_text(
        get_text(user_id, 'language_selected'),
        parse_mode="Markdown"
    )
    await callback.answer()

@dp.message(lambda message: message.text == "/language")
async def change_language(message: types.Message):
    user_id = message.from_user.id
    current_lang = user_languages.get(user_id, 'en')
    
    await message.answer(
        texts[current_lang]['select_language'],
        reply_markup=get_language_keyboard()
    )

@dp.message(lambda message: message.text == "/next")
async def next_partner(message: types.Message):
    user_id = message.from_user.id
    
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    user_lang = user_languages[user_id]
    
    # Remove from current chat
    if user_id in users:
        partner_id = users[user_id]
        if partner_id in users:
            del users[partner_id]
            await bot.send_message(partner_id, get_text(partner_id, 'partner_left'))
        del users[user_id]
    
    # Remove from all queues first
    for lang_queue in queues.values():
        if user_id in lang_queue:
            lang_queue.remove(user_id)
    
    # Add to appropriate language queue
    if user_id not in queues[user_lang]:
        queues[user_lang].append(user_id)
    
    await find_partner(user_id)

@dp.message(lambda message: message.text == "/stop")
async def stop_chat(message: types.Message):
    user_id = message.from_user.id
    
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    if user_id in users:
        partner_id = users[user_id]
        del users[user_id]
        if partner_id in users:
            del users[partner_id]
            await bot.send_message(partner_id, get_text(partner_id, 'partner_left'))
        await message.answer(get_text(user_id, 'chat_stopped'))
    else:
        await message.answer(get_text(user_id, 'not_in_chat'))

async def find_partner(user_id):
    user_lang = user_languages[user_id]
    
    if len(queues[user_lang]) >= 2:
        queues[user_lang].remove(user_id)
        partner_id = queues[user_lang].pop(0)
        
        # Connect users
        users[user_id] = partner_id
        users[partner_id] = user_id
        
        await bot.send_message(user_id, get_text(user_id, 'connected'))
        await bot.send_message(partner_id, get_text(partner_id, 'connected'))
    else:
        await bot.send_message(user_id, get_text(user_id, 'searching'))

@dp.message()
async def handle_message(message: types.Message):
    user_id = message.from_user.id
    
    # Check if user has selected language
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    # If user has partner, forward message
    if user_id in users:
        partner_id = users[user_id]
        
        try:
            if message.text:
                await bot.send_message(partner_id, message.text)
            elif message.photo:
                await bot.send_photo(partner_id, message.photo[-1].file_id, caption=message.caption)
            elif message.video:
                await bot.send_video(partner_id, message.video.file_id, caption=message.caption)
            elif message.voice:
                await bot.send_voice(partner_id, message.voice.file_id)
            elif message.sticker:
                await bot.send_sticker(partner_id, message.sticker.file_id)
            elif message.document:
                await bot.send_document(partner_id, message.document.file_id, caption=message.caption)
            elif message.audio:
                await bot.send_audio(partner_id, message.audio.file_id, caption=message.caption)
        except Exception as e:
            print(f"Error forwarding message: {e}")
            # Partner blocked bot or deleted account
            if user_id in users:
                del users[user_id]
            await message.answer(get_text(user_id, 'partner_left'))
    else:
        # Find partner for user
        user_lang = user_languages[user_id]
        
        # Remove from all queues first
        for lang_queue in queues.values():
            if user_id in lang_queue:
                lang_queue.remove(user_id)
        
        # Add to appropriate language queue
        if user_id not in queues[user_lang]:
            queues[user_lang].append(user_id)
        
        await find_partner(user_id)

async def main():
    # Database setup
    try:
        conn = await get_db()
        await conn.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id BIGINT PRIMARY KEY,
                username TEXT,
                first_name TEXT,
                language TEXT NOT NULL DEFAULT 'en',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        await conn.close()
        print("✅ Database tables created/verified")
    except Exception as e:
        print(f"❌ Database error: {e}")
        print("⚠️ Bot will work without database features")
    
    print("🚀 Multi-language Anonymous Chat Bot Started!")
    print("📊 Supported languages: 🇺🇿 O'zbek, 🇷🇺 Русский, 🇺🇸 English")
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'aiogram'

In [ ]:
import cv2
import numpy as np

def main():
    # Kamera yoki video faylni ochish
    cap = cv2.VideoCapture(0)  # 0 - birinchi kamera, fayl uchun fayl yo'lini bering
    
    # Haar cascade classifier yuklab olish (yuz aniqlash uchun)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Kamera ochilishini tekshirish
    if not cap.isOpened():
        print("Xato: Kamerani ochib bo'lmadi!")
        return
    
    print("Dastur ishga tushdi. 'q' tugmasini bosing chiqish uchun.")
    
    while True:
        # Kadrni o'qish
        ret, frame = cap.read()
        
        if not ret:
            print("Kadr o'qib bo'lmadi!")
            break
        
        # Kadrni oyna o'lchamiga moslash
        frame = cv2.resize(frame, (800, 600))
        
        # Kulrang rangga o'tkazish (yuz aniqlash uchun)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Yuzlarni aniqlash
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,        # Har bir darajada tasvirning qanchalik kichraytirilishi
            minNeighbors=5,         # Har bir nomzod to'rtburchak qancha qo'shni to'rtburchaklarga ega bo'lishi kerak
            minSize=(30, 30),       # Minimal yuz o'lchami
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        # Har bir aniqlangan yuz uchun
        for (x, y, w, h) in faces:
            # Yuz mintaqasini ajratib olish
            face_roi = frame[y:y+h, x:x+w]
            
            # Kuchli blur effekti qo'llash
            # Gaussian blur yoki motion blur ishlatishingiz mumkin
            blurred_face = cv2.GaussianBlur(face_roi, (99, 99), 30)
            
            # Alternativ: Motion blur
            # kernel_size = 50
            # motion_blur_kernel = np.zeros((kernel_size, kernel_size))
            # motion_blur_kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
            # motion_blur_kernel = motion_blur_kernel / kernel_size
            # blurred_face = cv2.filter2D(face_roi, -1, motion_blur_kernel)
            
            # Blur qilingan yuzni asl kadrga qaytarish
            frame[y:y+h, x:x+w] = blurred_face
            
            # Yuz atrofida to'rtburchak chizish (ixtiyoriy)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Matn qo'shish (ixtiyoriy)
            cv2.putText(frame, 'Yuz aniqlandi', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Natijani ko'rsatish
        cv2.imshow('Yuz Aniqlash va Blur', frame)
        
        # 'q' tugmasi bosilganda chiqish
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Resurslarni tozalash
    cap.release()
    cv2.destroyAllWindows()

# Dasturni ishga tushirish
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'cv2'

In [ ]:
def chiziqli_jarayon():
    print("\n--- 1. Chiziqli jarayon (Z) ---")
    try:
        
        k = float(input("k ni kiriting (masalan, 3.5): "))
        c = float(input("c ni kiriting (masalan, 1.0): "))
        m = float(input("m ni kiriting (masalan, 5.0): "))

        print(f"Berilganlar: k={k}, c={c}, m={m}")

        if c == 0:
            return "Hisoblash xatosi: c nolga teng."

        a = (2 * m) / c
        b = k * c

        if b == 0:
            return "Hisoblash xatosi: b nolga teng."

        Z = ((a + c) / b) ** 2

        print(f"a = {a:.4f}, b = {b:.4f}")
        return f"Natija Z: {Z:.4f}"
    except ValueError:
        return "Xatolik: Son kiritilmadi. Iltimos, raqam kiriting."

In [ ]:
import math

def chiziqli_jarayon():
    # Noma'lum o'zgaruvchilarga qiymat berish/kiritish
    k = float(input("k ni kiriting: "))
    c = float(input("c ni kiriting: "))
    m = float(input("m ni kiriting: "))
    
    print(f"\nBerilganlar: k={k}, c={c}, m={m}")
    
    if c == 0:
        return "Hisoblash xatosi: c nolga teng bo'lishi mumkin emas (a ni hisoblashda)."
    
    # Hisoblash
    a = (2 * m) / c
    b = k * c
    
    if b == 0:
        return "Hisoblash xatosi: b nolga teng bo'lishi mumkin emas (Z ni hisoblashda)."
        
    Z = ((a + c) / b) ** 2
    
    print(f"a = {a:.4f}, b = {b:.4f}")
    return f"Natija Z: {Z:.4f}"

# chiziqli_jarayon()

In [ ]:
import math

def chiziqli_jarayon(k, c, m):
    
    try:
        
        if c == 0:
            return "Xatolik: c nolga teng bo'lishi mumkin emas (a ni hisoblashda)"
        a = (2 * m) / c
        
       
        b = k * c
        
        # 3. Z ni hisoblash
        if b == 0:
            return "Xatolik: b nolga teng bo'lishi mumkin emas (Z ni hisoblashda)"
            
        Z = ((a + c) / b) ** 2
        
        return Z
    except Exception as e:
        return f"Hisoblashda xatolik yuz berdi: {e}"


k_val = 3.5
c_val = 1.0
m_val = 5.0
natija_z = chiziqli_jarayon(k_val, c_val, m_val)

print("--- Chiziqli jarayon ---")
print("Berilganlar: k={k_val}, c={c_val}, m={m_val}")
print("Natija Z: {natija_z}")

--- Chiziqli jarayon ---
Berilganlar: k={k_val}, c={c_val}, m={m_val}
Natija Z: {natija_z}


In [ ]:
def tarmoqlanuvchi_jarayon_1(x):
    
    if x < 0:
        b = x**2 - 7*x - 12
        return  {b}
    else: # x >= 0
        denominator = x**2 - 7*x - 12
        if denominator == 0:
            return c
        c = 3 / denominator
        return {c}

x1 = -5
x2 = 8
print("\n--- Tarmoqlanuvchi jarayon 1 ---")
print("x = {x1} uchun: {tarmoqlanuvchi_jarayon_1(x1)}")
print("x = {x2} uchun: {tarmoqlanuvchi_jarayon_1(x2)}")


--- Tarmoqlanuvchi jarayon 1 ---
x = {x1} uchun: {tarmoqlanuvchi_jarayon_1(x1)}
x = {x2} uchun: {tarmoqlanuvchi_jarayon_1(x2)}


In [ ]:
import math

def tarmoqlanuvchi_jarayon_2(x):
    
  
    
    if x < 0:
        y = math.sqrt(abs(x) + 7)
    else: # x >= 0
        y = x**2 + 3*x
    return y


x3 = -9
x4 = 2
print(f"\n--- Tarmoqlanuvchi jarayon 2 (y) ---")
print(f"x = {x3} uchun y: {tarmoqlanuvchi_jarayon_2(x3)}")
print(f"x = {x4} uchun y: {tarmoqlanuvchi_jarayon_2(x4)}")


--- Tarmoqlanuvchi jarayon 2 (y) ---
x = -9 uchun y: 4.0
x = 2 uchun y: 10


In [ ]:
import math

def tarmoqlanuvchi_jarayon_3(x, a, b, c):
    
    try:
        if x < -3:
            z = 16 * x**2 + 9 * x - 1
        elif -3 <= x <= 0:
            z = a + b * math.cos(x) 
        else: 
            argument = a**2 - b**2 * x + c
            if argument < 0:
                return f"Xatolik: Ildiz ostidagi ifoda manfiy bo'ldi ({argument})"
            z = math.sqrt(argument)
        return z
    except Exception as e:
        return f"{e}"


x5 = -4
x6 = -1.5
x7 = 5
a_val, b_val, c_val = 10, 2, 5
print(f"\n---  (z) ---")
print(f"a={a_val}, b={b_val}, c={c_val}")
print(f"x = {x5}  z: {tarmoqlanuvchi_jarayon_3(x5, a_val, b_val, c_val)}")
print(f"x = {x6}  z: {tarmoqlanuvchi_jarayon_3(x6, a_val, b_val, c_val)}")
print(f"x = {x7}  z: {tarmoqlanuvchi_jarayon_3(x7, a_val, b_val, c_val)}")


---  (z) ---
a=10, b=2, c=5
x = -4  z: 219
x = -1.5  z: 10.141474403335407
x = 5  z: 9.219544457292887


In [ ]:
print(************)
print(*******)
print(******)
print(****)
print(***)
print(***)p
print(**)
print(*)
print(**)
print(***)
print(****)


SyntaxError: invalid syntax (2268304873.py, line 1)

In [ ]:
#!/usr/bin/env python3
# rainbow_balls.py
# Terminalda rangli sharchalar sakrab yuradi. Chiqish: 'q' tugmasi

import curses
import random
import time

class Ball:
    def __init__(self, max_y, max_x, color_pair):
        self.y = random.uniform(1, max_y-2)
        self.x = random.uniform(1, max_x-2)
        self.vy = random.uniform(-0.8, 0.8)
        self.vx = random.uniform(-1.2, 1.2)
        self.char = random.choice(['o','●','@','*'])
        self.color = color_pair

    def step(self, max_y, max_x):
        self.y += self.vy
        self.x += self.vx
        # zamin va shift tekshiruvi — to‘qnashganda qaytish
        if self.y < 1:
            self.y = 1
            self.vy *= -1
        if self.y > max_y-2:
            self.y = max_y-2
            self.vy *= -0.9
        if self.x < 1:
            self.x = 1
            self.vx *= -1
        if self.x > max_x-2:
            self.x = max_x-2
            self.vx *= -1

def main(stdscr):
    curses.curs_set(0)
    stdscr.nodelay(True)
    stdscr.timeout(0)

    # Ranglarni ishga tushuramiz (agar terminal qo'llab-quvvatlasa)
    if curses.has_colors():
        curses.start_color()
        curses.use_default_colors()
        # ba'zi kombinatsiyalarni ro'yxatlab olamiz
        pairs = []
        color_ids = [curses.COLOR_RED, curses.COLOR_GREEN, curses.COLOR_YELLOW,
                     curses.COLOR_BLUE, curses.COLOR_MAGENTA, curses.COLOR_CYAN, curses.COLOR_WHITE]
        for i, col in enumerate(color_ids, start=1):
            try:
                curses.init_pair(i, col, -1)
                pairs.append(i)
            except:
                pass
    else:
        pairs = [0]

    max_y, max_x = stdscr.getmaxyx()
    n_balls = max(6, min(25, max_x // 6))  # terminalga moslab sharchalar soni

    balls = [Ball(max_y, max_x, random.choice(pairs)) for _ in range(n_balls)]
    last_resize_check = time.time()

    while True:
        # o'lcham o'zgargan-yo'qligini tekshirish va moslashish (har 0.5s)
        if time.time() - last_resize_check > 0.5:
            new_y, new_x = stdscr.getmaxyx()
            if new_y != max_y or new_x != max_x:
                max_y, max_x = new_y, new_x
            last_resize_check = time.time()

        stdscr.erase()
        # har bir sharni yangilash va chizish
        for b in balls:
            b.step(max_y, max_x)
            try:
                if b.color != 0:
                    stdscr.addstr(int(b.y), int(b.x), b.char, curses.color_pair(b.color))
                else:
                    stdscr.addstr(int(b.y), int(b.x), b.char)
            except curses.error:
                # ba'zida terminal chegarasida xato bo'ladi — e'tibor bermaymiz
                pass

        # yordamchi satr
        hint = "Press 'q' to quit — Rainbow Balls"
        try:
            stdscr.addstr(0, max(0, (max_x - len(hint))//2), hint, curses.A_DIM)
        except curses.error:
            pass

        stdscr.refresh()

        # sekinlashtirish va tugma tekshiruvi
        time.sleep(0.05)
        ch = stdscr.getch()
        if ch == ord('q'):
            break
        # bitta tugma bilan yangi shar qo'shish (masalan: space)
        if ch == ord(' '):
            balls.append(Ball(max_y, max_x, random.choice(pairs)))
            if len(balls) > 60:
                balls = balls[-60:]

if __name__ == "__main__":
    try:
        curses.wrapper(main)
    except KeyboardInterrupt:
        pass


)07>

error: nocbreak() returned ERR

In [ ]:
import turtle
import random
import colorsys

t = turtle.Turtle()
t.speed(0)
t.width(2)
t.hideturtle()
turtle.bgcolor("black")

# Rangli aylanish (gradient)
hue = 0
turtle.tracer(50)
for i in range(360):
    color = colorsys.hsv_to_rgb(hue, 1, 1)
    t.pencolor(color)
    t.circle(100 + i, 60)
    t.right(61)
    hue += 0.005

turtle.done()


KeyboardInterrupt: 

In [ ]:
# save as public_number_search.py
# Requires: pip install requests
# Setup: get API_KEY and CX (Custom Search Engine ID) from Google Cloud Console / Custom Search.
import requests
import sys
import urllib.parse

API_KEY = "YOUR_API_KEY"   # o'zingiz o'rnating
CX = "YOUR_CX"             # o'zingiz o'rnating

def search_number(number, num_results=10):
    q = f'"{number}"'  # raqamni qo'shtirnoqda qidirish - aniq moslik
    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": API_KEY, "cx": CX, "q": q, "num": min(10, num_results)}
    resp = requests.get(url, params=params, timeout=15)
    resp.raise_for_status()
    data = resp.json()
    items = data.get("items", [])
    results = []
    for it in items:
        results.append({
            "title": it.get("title"),
            "snippet": it.get("snippet"),
            "link": it.get("link")
        })
    return results

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python3 public_number_search.py +998901234567")
        sys.exit(1)
    number = sys.argv[1].strip()
    print(f"Searching public web for: {number}")
    try:
        res = search_number(number)
        if not res:
            print("No public results found (via Google Custom Search).")
        else:
            for i, r in enumerate(res, 1):
                print(f"\n[{i}] {r['title']}\n{r['snippet']}\n{r['link']}")
    except Exception as e:
        print("Error:", e)


In [ ]:
#!/usr/bin/env python3
# show_my_face.py
# Webcamni ochadi, yuzni aniqlaydi va oynada ko'rsatadi.
# 's' - snapshot saqlash, 'q' - chiqish

import cv2
import time
import os

# Yuz uchun Haar cascade fayli (OpenCV paketidagi yo'l)
cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(cascade_path)

if face_cascade.empty():
    raise RuntimeError("Haar cascade yuklanmadi. OpenCV to'liq o'rnatilganligini tekshiring.")

# Kamera: 0 — birinchi (odatda ichki) kamera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera ochilmadi. Kamera ulangan va ruxsatlar berilganligini tekshiring.")

# Oyna nomi
win_name = "Sizning yuzingiz - Press 's' to save, 'q' to quit"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

snapshot_dir = "snapshots"
os.makedirs(snapshot_dir, exist_ok=True)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kamera oqimi yo'q — qayta urinib ko'rilyapti...")
            time.sleep(0.5)
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Yuzlarni aniqlash (scaleFactor, minNeighbors ni siz sozlashingiz mumkin)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Har bir yuz atrofiga to'rtburchak chizish
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 200, 0), 2)
            # Yuz tepasiga label qo'yish (ixtiyoriy)
            cv2.putText(frame, "Yuz", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,200,0), 2)

        # Ko'rsatish
        cv2.imshow(win_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            t = time.strftime("%Y%m%d-%H%M%S")
            fname = os.path.join(snapshot_dir, f"snapshot_{t}.png")
            cv2.imwrite(fname, frame)
            print(f"Snapshot saqlandi: {fname}")

finally:
    cap.release()
    cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [ ]:
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, filters
import logging

# 1. Logging sozlamalari (Ixtiyoriy, xatolarni ko'rish uchun foydali)
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# 2. Bot tokeni. Uni @BotFather'dan olasiz.
# Eslatma: Haqiqiy tokenni shu yerga qo'ying!
TOKEN = "BU_YERGA_O'ZINGIZNING_TOKENINGIZNI_QO'YING" 

# 3. /start buyrug'i uchun funksiya
def start(update, context):
    """Foydalanuvchi /start buyrug'ini yuborganda ishga tushadi"""
    update.message.reply_text('Salom! Men oddiy Python botiman. Menga biron nima yozib ko‘ring.')

# 4. Foydalanuvchi matn yuborganda javob beruvchi funksiya
def echo(update, context):
    """Foydalanuvchining xabarini qabul qiladi va aks-sado (echo) qiladi"""
    user_text = update.message.text
    response_text = f"Siz menga yozdingiz: *{user_text}*"
    
    # Matnni javob tariqasida qaytarish (Markdown formatida)
    update.message.reply_text(response_text, parse_mode=telegram.ParseMode.MARKDOWN)

# 5. Xatolarni ushlab oluvchi funksiya (Ixtiyoriy)
def error(update, context):
    """Updater tomonidan yuzaga kelgan xatolarni jurnalga yozadi"""
    logging.warning(f'Update "%s" xato keltirdi: "%s"', update, context.error)

# 6. Asosiy funksiya - Botni ishga tushirish
def main():
    # Updater ob'ektini yaratish va token ulab ishlashga tayyorlash
    updater = Updater(TOKEN, use_context=True)

    # Dispatcher'ga buyruqlar va xabar ishlovchilarni qo'shish
    dp = updater.dispatcher

    # /start buyrug'ini ishlovchisini qo'shish
    dp.add_handler(CommandHandler("start", start))

    # Matn xabarlarini ishlovchisini qo'shish (matnli bo'lmagan xabarlarni e'tiborsiz qoldiradi)
    dp.add_handler(MessageHandler(filters.Text(), echo))

    # Xato ishlovchisini qo'shish
    dp.add_error_handler(error)

    # Botni ishga tushirish (polling rejimida)
    updater.start_polling()

    # Ctrl+C bosilguncha ishlashni davom ettirish
    updater.idle()

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'telegram'

In [ ]:
import random
import string

def generate_password(length=12):
    """
    Belgilangan uzunlikda (default 12) tasodifiy mustahkam parol yaratadi.
    """
    # Parol uchun ishlatiladigan barcha belgilar to'plami
    characters = string.ascii_letters + string.digits + string.punctuation
    
    # Parolning minimum talablarini qondirish (Kamida 1 ta katta harf, 1 ta kichik harf, 1 ta raqam, 1 ta belgi)
    password = [
        random.choice(string.ascii_uppercase),
        random.choice(string.ascii_lowercase),
        random.choice(string.digits),
        random.choice(string.punctuation)
    ]
    
    # Qolgan uzunlikni tasodifiy belgilar bilan to'ldirish
    remaining_length = length - len(password)
    if remaining_length > 0:
        password += random.choices(characters, k=remaining_length)
    
    # Paroldagi barcha belgilarni aralashtirish
    random.shuffle(password)
    
    # Ro'yxatni (list) bitta satrga (string) birlashtirish
    return "".join(password)

# Misol uchun ishlatish
parol_12 = generate_password(12)
parol_16 = generate_password(16)

print(f"12 belgili tasodifiy parol: {parol_12}")
print(f"16 belgili tasodifiy parol: {parol_16}")

# Natija har safar yangi, tasodifiy bo'ladi!


12 belgili tasodifiy parol: "[\3ZE$'q{Hi
16 belgili tasodifiy parol: hFNl(gZ'7p[9Hg/+


In [ ]:
import psutil
import platform

def convert_bytes_to_gb(bytes_value):
    """Baytlarni Gigabayt (GB) ga o'tkazuvchi yordamchi funksiya"""
    return round(bytes_value / (1024 ** 3), 2)

def show_memory_info():
    print("="*40, "Xotira (RAM) ma'lumotlari", "="*40)
    
    # 1. RAM (Tezkor xotira) ma'lumotlari
    memory = psutil.virtual_memory()
    total_ram = convert_bytes_to_gb(memory.total)
    used_ram = convert_bytes_to_gb(memory.used)
    available_ram = convert_bytes_to_gb(memory.available)
    ram_percent = memory.percent
    
    print(f"Umumiy RAM (Operativ xotira): {total_ram} GB")
    print(f"Ishlatilgan RAM:               {used_ram} GB ({ram_percent}%)")
    print(f"Bo'sh/Mavjud RAM:              {available_ram} GB")

    print("\n" + "="*40, "Disk (Saqlash) xotirasi ma'lumotlari", "="*40)
    
    # 2. Disk (Qattiq disk/SSD) ma'lumotlari
    # Windowsda odatda 'C:\', Linux/macOS da '/' (root)
    if platform.system() == "Windows":
        path = 'C:\\'
    else:
        path = '/'
        
    disk = psutil.disk_usage(path)
    total_disk = convert_bytes_to_gb(disk.total)
    used_disk = convert_bytes_to_gb(disk.used)
    free_disk = convert_bytes_to_gb(disk.free)
    disk_percent = disk.percent
    
    print(f"Tekshirilgan disk:             {path}")
    print(f"Umumiy Disk hajmi:             {total_disk} GB")
    print(f"Ishlatilgan Disk:              {used_disk} GB ({disk_percent}%)")
    print(f"Bo'sh Disk hajmi:              {free_disk} GB")

# Funksiyani chaqirish
show_memory_info()

======================================== Xotira (RAM) ma'lumotlari ========================================
Umumiy RAM (Operativ xotira): 7.49 GB
Ishlatilgan RAM:               4.79 GB (77.6%)
Bo'sh/Mavjud RAM:              1.68 GB

======================================== Disk (Saqlash) xotirasi ma'lumotlari ========================================
Tekshirilgan disk:             /
Umumiy Disk hajmi:             467.35 GB
Ishlatilgan Disk:              40.28 GB (9.1%)
Bo'sh Disk hajmi:              403.26 GB


In [ ]:
print("****")

****


In [ ]:
import phonenumbers
from phonenumbers import geocoder, carrier, timezone

# 1. Tekshiriladigan telefon raqamini E.164 formatida kiriting.
# Ya'ni, '+' belgisidan so'ng mamlakat kodi va to'liq raqam.
# Misol: O'zbekiston raqami: +998901234567
phone_number_str = input("Iltimos, tekshirmoqchi bo'lgan telefon raqamni kiriting (Masalan, +998901234567): ")

try:
    # Raqamni tahlil qilish (parse)
    parsed_number = phonenumbers.parse(phone_number_str)

    # Raqamning haqiqiyligini tekshirish
    is_valid = phonenumbers.is_valid_number(parsed_number)
    
    if not is_valid:
        print("\nDIQQAT: Kiritilgan raqam formati noto'g'ri yoki haqiqiy emas.")
        print(f"Raqam formati to'g'rimi?: {phonenumbers.is_possible_number(parsed_number)}")
    else:
        print("\n" + "="*40)
        print(f"Telefon raqami: {phone_number_str}")
        print(f"Haqiqiyligi:    {'HAQIQIY' if is_valid else 'NOQONUNIY'}")
        print("="*40)
        
        # 2. Geografik joylashuv (Mamlakat)
        location = geocoder.description_for_number(parsed_number, "uz") # 'uz' tilida chiqarishga harakat qiladi
        print(f"Taxminiy joylashuv: {location}")

        # 3. Vaqt mintaqasi
        time_zones = timezone.time_zones_for_number(parsed_number)
        print(f"Vaqt mintaqasi:     {', '.join(time_zones)}")

        # 4. Aloqa operatori
        service_provider = carrier.name_for_number(parsed_number, "uz")
        print(f"Aloqa operatori:    {service_provider}")
        print("="*40)
        
except phonenumbers.phonenumberutil.NumberParseException as e:
    print(f"\nKiritishda xato: {e}. Raqamni '+998XXXXXXXXX' formatida kiriting.")
except Exception as e:
    print(f"\nKutilmagan xato yuz berdi: {e}")

ModuleNotFoundError: No module named 'phonenumbers'

In [ ]:
import random, time, os
from colorama import Fore, Style, init
init()

chars = "01"
while True:
    print(Fore.GREEN + "".join(random.choice(chars) for _ in range(80)))
    time.sleep(0.05)


01000110101111010001110011101001101110010010010000000111101101010110001110101101
00101001010000011100010111011010111001101111011011011101111111011111110010100100
01010000101010001100110100100111111101011000111011011011001010100010111001000010
10010000111110100011100000000001011101111000010001001110100110000101010010000011
10100101101100000100001010111101110101001010101011001011011110110100011110010110
10001110101001010001001110110100010011110001001111111000100110101110001100010100
00011011011111101011010000101010101011101100110110111100111101101001111111001001
00100111001011100111000100000100010011001001111001011110111011010111010111001001
11100110110101011111101101001110011100110010000000111011000010100101010000011010
00010010010101100101010000101101000101110010111001101000110111101011001010111001
00010001100011111010101110001100110101101011011000110000001111000111011011101111
11110101001011010000010101010110101100011001000110010100010000101010110101000001
0001111110101100011000010110

KeyboardInterrupt: 

In [1]:
import pygame
import random
import math

# Pygame initsializatsiya
pygame.init()

# Ekran o'lchamlari
WIDTH, HEIGHT = 1000, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("MOBA Legends - Mobile Legends Style")

# Ranglar
BLACK = (26, 26, 46)
WHITE = (236, 240, 241)
RED = (231, 76, 60)
GREEN = (46, 204, 113)
BLUE = (52, 152, 219)
PURPLE = (155, 89, 182)
YELLOW = (241, 196, 15)
ORANGE = (230, 126, 34)
GRAY = (149, 165, 166)
DARK_RED = (192, 57, 43)
DARK_GREEN = (39, 174, 96)

# FPS
clock = pygame.time.Clock()
FPS = 60

# Shriftlar
font_small = pygame.font.Font(None, 24)
font_medium = pygame.font.Font(None, 36)
font_large = pygame.font.Font(None, 48)
font_title = pygame.font.Font(None, 72)


class Particle:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.vx = random.uniform(-3, 3)
        self.vy = random.uniform(-3, 3)
        self.life = 30
        self.color = color
        self.size = random.randint(2, 5)

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.vx *= 0.95
        self.vy *= 0.95
        self.life -= 1

    def draw(self, surface):
        alpha = int(255 * (self.life / 30))
        s = pygame.Surface((self.size * 2, self.size * 2), pygame.SRCALPHA)
        color_with_alpha = (*self.color, alpha)
        pygame.draw.circle(s, color_with_alpha, (self.size, self.size), self.size)
        surface.blit(s, (int(self.x - self.size), int(self.y - self.size)))


class Projectile:
    def __init__(self, x, y, target_x, target_y, damage, color, owner):
        self.x = x
        self.y = y
        self.damage = damage
        self.color = color
        self.owner = owner
        self.size = 8
        
        angle = math.atan2(target_y - y, target_x - x)
        speed = 10
        self.vx = math.cos(angle) * speed
        self.vy = math.sin(angle) * speed

    def update(self):
        self.x += self.vx
        self.y += self.vy

    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), self.size)
        # Glow effect
        s = pygame.Surface((self.size * 4, self.size * 4), pygame.SRCALPHA)
        pygame.draw.circle(s, (*self.color, 50), (self.size * 2, self.size * 2), self.size * 2)
        surface.blit(s, (int(self.x - self.size * 2), int(self.y - self.size * 2)))

    def is_off_screen(self):
        return self.x < 0 or self.x > WIDTH or self.y < 0 or self.y > HEIGHT


class Tower:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.max_hp = 500
        self.hp = 500
        self.size = 45
        self.damage = 35
        self.attack_cooldown = 0
        self.attack_range = 250

    def update(self, targets, projectiles, particles):
        if self.hp <= 0:
            return
        
        self.attack_cooldown = max(0, self.attack_cooldown - 1)
        
        if self.attack_cooldown == 0:
            # Eng yaqin dushmanni topish
            nearest = None
            min_dist = self.attack_range
            
            for target in targets:
                if hasattr(target, 'hp') and target.hp > 0:
                    dist = math.hypot(target.x - self.x, target.y - self.y)
                    if dist < min_dist:
                        min_dist = dist
                        nearest = target
            
            if nearest:
                projectiles.append(Projectile(
                    self.x, self.y, nearest.x, nearest.y,
                    self.damage,
                    GREEN if self.team == 'player' else RED,
                    self
                ))
                self.attack_cooldown = 60

    def draw(self, surface):
        if self.hp <= 0:
            return
        
        # Tower body
        color = DARK_GREEN if self.team == 'player' else DARK_RED
        pygame.draw.rect(surface, color, 
                        (self.x - self.size//2, self.y - self.size//2, self.size, self.size))
        pygame.draw.rect(surface, WHITE, 
                        (self.x - self.size//2, self.y - self.size//2, self.size, self.size), 3)
        
        # HP bar
        bar_width = 60
        bar_height = 8
        bar_x = self.x - bar_width // 2
        bar_y = self.y - self.size - 15
        
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, bar_width, bar_height))
        hp_width = int(bar_width * (self.hp / self.max_hp))
        pygame.draw.rect(surface, GREEN, (bar_x, bar_y, hp_width, bar_height))


class Minion:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.max_hp = 60
        self.hp = 60
        self.size = 15
        self.damage = 10
        self.speed = 1.2
        self.attack_cooldown = 0
        self.target = None

    def find_target(self, enemies, minions, towers):
        targets = []
        if self.team == 'player':
            targets = [e for e in enemies if e.hp > 0]
            targets.extend([m for m in minions if m.team == 'enemy' and m.hp > 0])
            targets.extend([t for t in towers if t.team == 'enemy' and t.hp > 0])
        else:
            targets = [m for m in minions if m.team == 'player' and m.hp > 0]
            targets.extend([t for t in towers if t.team == 'player' and t.hp > 0])
        
        if not targets:
            return None
        
        # Eng yaqinini topish
        nearest = min(targets, key=lambda t: math.hypot(t.x - self.x, t.y - self.y))
        return nearest

    def update(self, enemies, minions, towers, projectiles):
        if self.hp <= 0:
            return
        
        self.attack_cooldown = max(0, self.attack_cooldown - 1)
        
        if not self.target or (hasattr(self.target, 'hp') and self.target.hp <= 0):
            self.target = self.find_target(enemies, minions, towers)
        
        if self.target:
            dx = self.target.x - self.x
            dy = self.target.y - self.y
            dist = math.sqrt(dx * dx + dy * dy)
            
            if dist > 60:
                self.x += (dx / dist) * self.speed
                self.y += (dy / dist) * self.speed
            elif self.attack_cooldown == 0:
                projectiles.append(Projectile(
                    self.x, self.y, self.target.x, self.target.y,
                    self.damage,
                    GREEN if self.team == 'player' else RED,
                    self
                ))
                self.attack_cooldown = 80

    def draw(self, surface):
        if self.hp <= 0:
            return
        
        color = GREEN if self.team == 'player' else RED
        pygame.draw.circle(surface, color, (int(self.x), int(self.y)), self.size)
        pygame.draw.circle(surface, WHITE, (int(self.x), int(self.y)), self.size, 2)
        
        # HP bar
        bar_width = 30
        bar_height = 4
        bar_x = self.x - bar_width // 2
        bar_y = self.y - self.size - 10
        
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, bar_width, bar_height))
        hp_width = int(bar_width * (self.hp / self.max_hp))
        pygame.draw.rect(surface, GREEN, (bar_x, bar_y, hp_width, bar_height))


class Enemy:
    def __init__(self, x, y, level):
        self.x = x
        self.y = y
        self.max_hp = 100 + level * 15
        self.hp = self.max_hp
        self.size = 22
        self.damage = 15 + level * 3
        self.speed = 1.5 + level * 0.1
        self.attack_cooldown = 0

    def update(self, player, projectiles):
        if self.hp <= 0:
            return
        
        self.attack_cooldown = max(0, self.attack_cooldown - 1)
        
        dx = player.x - self.x
        dy = player.y - self.y
        dist = math.sqrt(dx * dx + dy * dy)
        
        if dist > 100:
            self.x += (dx / dist) * self.speed
            self.y += (dy / dist) * self.speed
        elif self.attack_cooldown == 0:
            projectiles.append(Projectile(
                self.x, self.y, player.x, player.y,
                self.damage, ORANGE, self
            ))
            self.attack_cooldown = 90

    def draw(self, surface):
        if self.hp <= 0:
            return
        
        pygame.draw.circle(surface, ORANGE, (int(self.x), int(self.y)), self.size)
        pygame.draw.circle(surface, WHITE, (int(self.x), int(self.y)), self.size, 2)
        
        # HP bar
        bar_width = 40
        bar_height = 5
        bar_x = self.x - bar_width // 2
        bar_y = self.y - self.size - 12
        
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, bar_width, bar_height))
        hp_width = int(bar_width * (self.hp / self.max_hp))
        pygame.draw.rect(surface, RED, (bar_x, bar_y, hp_width, bar_height))


class Player:
    def __init__(self, hero):
        self.x = 150
        self.y = HEIGHT // 2
        self.hero = hero
        self.max_hp = hero['hp']
        self.hp = self.max_hp
        self.damage = hero['damage']
        self.speed = hero['speed']
        self.color = hero['color']
        self.size = 28
        self.level = 1
        self.exp = 0
        self.kills = 0
        self.skill_cooldown = 0

    def move(self, keys):
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            self.x = max(self.size, self.x - self.speed)
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            self.x = min(WIDTH - self.size, self.x + self.speed)
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            self.y = max(self.size, self.y - self.speed)
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            self.y = min(HEIGHT - self.size, self.y + self.speed)

    def use_skill(self, enemies, projectiles, particles):
        if self.skill_cooldown > 0:
            return False
        
        # Eng yaqin dushmanni topish
        nearest = None
        min_dist = 400
        
        for enemy in enemies:
            if enemy.hp > 0:
                dist = math.hypot(enemy.x - self.x, enemy.y - self.y)
                if dist < min_dist:
                    min_dist = dist
                    nearest = enemy
        
        if nearest:
            projectiles.append(Projectile(
                self.x, self.y, nearest.x, nearest.y,
                self.damage * 2, self.color, self
            ))
            self.skill_cooldown = 180
            
            # Particles
            for _ in range(20):
                particles.append(Particle(self.x, self.y, self.color))
            return True
        return False

    def gain_exp(self, amount, particles):
        self.exp += amount
        if self.exp >= self.level * 100:
            self.level += 1
            self.exp = 0
            self.max_hp += 25
            self.hp = self.max_hp
            self.damage += 5
            
            # Level up particles
            for _ in range(40):
                particles.append(Particle(self.x, self.y, YELLOW))

    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), self.size)
        pygame.draw.circle(surface, WHITE, (int(self.x), int(self.y)), self.size, 3)
        
        # HP bar
        bar_width = 60
        bar_height = 8
        bar_x = self.x - bar_width // 2
        bar_y = self.y - self.size - 15
        
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, bar_width, bar_height))
        hp_width = int(bar_width * (self.hp / self.max_hp))
        pygame.draw.rect(surface, GREEN, (bar_x, bar_y, hp_width, bar_height))
        
        # Skill cooldown indicator
        if self.skill_cooldown > 0:
            angle = 2 * math.pi * (1 - self.skill_cooldown / 180)
            points = [(self.x, self.y)]
            for i in range(int(angle * 10) + 1):
                a = -math.pi/2 + (i / 10)
                px = self.x + math.cos(a) * (self.size + 7)
                py = self.y + math.sin(a) * (self.size + 7)
                points.append((px, py))
            points.append((self.x, self.y))
            
            s = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            pygame.draw.polygon(s, (*BLUE, 100), points)
            surface.blit(s, (0, 0))


def draw_menu(screen, heroes, selected_hero_idx):
    screen.fill(BLACK)
    
    # Title
    title = font_title.render("MOBA LEGENDS", True, YELLOW)
    title_rect = title.get_rect(center=(WIDTH // 2, 80))
    screen.blit(title, title_rect)
    
    subtitle = font_medium.render("Mobile Legends Style", True, WHITE)
    subtitle_rect = subtitle.get_rect(center=(WIDTH // 2, 140))
    screen.blit(subtitle, subtitle_rect)
    
    # Hero selection
    text = font_large.render("QAHRAMONINGIZNI TANLANG", True, WHITE)
    text_rect = text.get_rect(center=(WIDTH // 2, 200))
    screen.blit(text, text_rect)
    
    # Heroes
    hero_y = 280
    for i, hero in enumerate(heroes):
        hero_x = WIDTH // 2 - 400 + i * 220
        
        # Hero card
        card_color = YELLOW if i == selected_hero_idx else GRAY
        pygame.draw.rect(screen, card_color, (hero_x, hero_y, 200, 240), 3)
        
        # Hero circle
        pygame.draw.circle(screen, hero['color'], (hero_x + 100, hero_y + 60), 40)
        pygame.draw.circle(screen, WHITE, (hero_x + 100, hero_y + 60), 40, 2)
        
        # Hero name
        name = font_medium.render(hero['name'], True, WHITE)
        name_rect = name.get_rect(center=(hero_x + 100, hero_y + 120))
        screen.blit(name, name_rect)
        
        # Stats
        stats_y = hero_y + 150
        hp_text = font_small.render(f"HP: {hero['hp']}", True, RED)
        dmg_text = font_small.render(f"DMG: {hero['damage']}", True, ORANGE)
        spd_text = font_small.render(f"SPD: {hero['speed']}", True, BLUE)
        
        screen.blit(hp_text, (hero_x + 20, stats_y))
        screen.blit(dmg_text, (hero_x + 20, stats_y + 25))
        screen.blit(spd_text, (hero_x + 20, stats_y + 50))
    
    # Instructions
    inst = font_small.render("1-4: Qahramon tanlash | SPACE: Boshlash", True, WHITE)
    inst_rect = inst.get_rect(center=(WIDTH // 2, 580))
    screen.blit(inst, inst_rect)
    
    controls = font_small.render("WASD: Harakat | SPACE: Skill", True, GRAY)
    controls_rect = controls.get_rect(center=(WIDTH // 2, 620))
    screen.blit(controls, controls_rect)


def main():
    heroes = [
        {'name': 'Warrior', 'hp': 180, 'damage': 28, 'speed': 3.5, 'color': RED},
        {'name': 'Mage', 'hp': 120, 'damage': 40, 'speed': 3, 'color': BLUE},
        {'name': 'Assassin', 'hp': 140, 'damage': 35, 'speed': 4.5, 'color': PURPLE},
        {'name': 'Tank', 'hp': 250, 'damage': 18, 'speed': 2.5, 'color': GRAY}
    ]
    
    selected_hero_idx = 0
    game_state = 'menu'
    
    running = True
    while running:
        if game_state == 'menu':
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_1:
                        selected_hero_idx = 0
                    elif event.key == pygame.K_2:
                        selected_hero_idx = 1
                    elif event.key == pygame.K_3:
                        selected_hero_idx = 2
                    elif event.key == pygame.K_4:
                        selected_hero_idx = 3
                    elif event.key == pygame.K_SPACE:
                        game_state = 'playing'
                        # O'yin o'zgaruvchilari
                        player = Player(heroes[selected_hero_idx])
                        enemies = []
                        minions = []
                        projectiles = []
                        particles = []
                        
                        towers = [
                            Tower(WIDTH - 120, HEIGHT // 3, 'enemy'),
                            Tower(WIDTH - 120, 2 * HEIGHT // 3, 'enemy'),
                            Tower(120, HEIGHT // 3, 'player'),
                            Tower(120, 2 * HEIGHT // 3, 'player')
                        ]
                        
                        enemy_spawn_timer = 0
                        minion_spawn_timer = 0
                        score = 0
            
            draw_menu(screen, heroes, selected_hero_idx)
        
        elif game_state == 'playing':
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        player.use_skill(enemies, projectiles, particles)
                    if event.key == pygame.K_ESCAPE:
                        game_state = 'menu'
            
            keys = pygame.key.get_pressed()
            player.move(keys)
            
            if player.skill_cooldown > 0:
                player.skill_cooldown -= 1
            
            # Spawn enemies
            enemy_spawn_timer += 1
            if enemy_spawn_timer > max(120 - player.level * 5, 60):
                enemies.append(Enemy(WIDTH - 80, random.randint(100, HEIGHT - 100), player.level))
                enemy_spawn_timer = 0
            
            # Spawn minions
            minion_spawn_timer += 1
            if minion_spawn_timer > 180:
                minions.append(Minion(80, HEIGHT // 2, 'player'))
                minions.append(Minion(WIDTH - 80, HEIGHT // 2, 'enemy'))
                minion_spawn_timer = 0
            
            # Update enemies
            for enemy in enemies[:]:
                enemy.update(player, projectiles)
                if enemy.hp <= 0:
                    enemies.remove(enemy)
                    player.gain_exp(25, particles)
                    player.kills += 1
                    score += 1
                    for _ in range(15):
                        particles.append(Particle(enemy.x, enemy.y, ORANGE))
            
            # Update minions
            for minion in minions[:]:
                minion.update(enemies, minions, towers, projectiles)
                if minion.hp <= 0:
                    minions.remove(minion)
            
            # Update towers
            for tower in towers:
                if tower.team == 'player':
                    targets = enemies + [m for m in minions if m.team == 'enemy']
                else:
                    targets = [player] + [m for m in minions if m.team == 'player']
                tower.update(targets, projectiles, particles)
            
            # Update projectiles
            for proj in projectiles[:]:
                proj.update()
                
                if proj.is_off_screen():
                    projectiles.remove(proj)
                    continue
                
                # Collision detection
                hit = False
                
                if proj.owner == player or isinstance(proj.owner, (Minion, Tower)) and \
                   (isinstance(proj.owner, Tower) and proj.owner.team == 'player' or \
                    isinstance(proj.owner, Minion) and proj.owner.team == 'player'):
                    # Player-side projectile
                    for enemy in enemies:
                        if enemy.hp > 0:
                            dist = math.hypot(proj.x - enemy.x, proj.y - enemy.y)
                            if dist < enemy.size + proj.size:
                                enemy.hp -= proj.damage
                                hit = True
                                for _ in range(8):
                                    particles.append(Particle(proj.x, proj.y, proj.color))
                                break
                    
                    if not hit:
                        for minion in minions:
                            if minion.team == 'enemy' and minion.hp > 0:
                                dist = math.hypot(proj.x - minion.x, proj.y - minion.y)
                                if dist < minion.size + proj.size:
                                    minion.hp -= proj.damage
                                    hit = True
                                    break
                else:
                    # Enemy projectile
                    dist = math.hypot(proj.x - player.x, proj.y - player.y)
                    if dist < player.size + proj.size:
                        player.hp -= proj.damage
                        hit = True
                        for _ in range(8):
                            particles.append(Particle(proj.x, proj.y, RED))
                        
                        if player.hp <= 0:
                            game_state = 'gameover'
                    
                    if not hit:
                        for minion in minions:
                            if minion.team == 'player' and minion.hp > 0:
                                dist = math.hypot(proj.x - minion.x, proj.y - minion.y)
                                if dist < minion.size + proj.size:
                                    minion.hp -= proj.damage
                                    hit = True
                                    break
                
                if hit:
                    projectiles.remove(proj)
            
            # Update particles
            for particle in particles[:]:
                particle.update()
                if particle.life <= 0:
                    particles.remove(particle)
            
            # Draw everything
            screen.fill(BLACK)
            
            # Draw lane
            pygame.draw.rect(screen, (40, 40, 60), (0, HEIGHT // 2 - 60, WIDTH, 120))
            
            # Draw towers
            for tower in towers:
                tower.draw(screen)
            
            # Draw minions
            for minion in minions:
                minion.draw(screen)
            
            # Draw enemies
            for enemy in enemies:
                enemy.draw(screen)
            
            # Draw projectiles
            for proj in projectiles:
                proj.draw(screen)
            
            # Draw particles
            for particle in particles:
                particle.draw(screen)
            
            # Draw player
            player.draw(screen)
            
            # HUD
            level_text = font_medium.render(f"Level: {player.level}", True, WHITE)
            screen.blit(level_text, (20, 20))
            
            kills_text = font_medium.render(f"Kills: {player.kills}", True, YELLOW)
            screen.blit(kills_text, (20, 55))
            
            hp_text = font_medium.render(f"HP: {max(0, int(player.hp))}/{player.max_hp}", True, RED)
            screen.blit(hp_text, (20, 90))
            
            exp_text = font_small.render(f"EXP: {player.exp}/{player.level * 100}", True, BLUE)
            screen.blit(exp_text, (20, 125))
            
            skill_status = "READY" if player.skill_cooldown == 0 else f"{player.skill_cooldown // 60 + 1}s"
            skill_color = GREEN if player.skill_cooldown == 0 else GRAY
            skill_text = font_medium.render(f"Skill: {skill_status}", True, skill_color)
            screen.blit(skill_text, (20, 155))
            
            # Score
            score_text = font_large.render(f"Score: {score}", True, YELLOW)
            score_rect = score_text.get_rect(center=(WIDTH // 2, 30))
            screen.blit(score_text, score_rect)
        
        elif game_state == 'gameover':
            screen.fill(BLACK)
            
            gameover_text = font_title.render("O'YIN TUGADI!", True, RED)
            gameover_rect = gameover_text.get_rect(center=(WIDTH // 2, HEIGHT // 2 - 100))
            screen.blit(gameover_text, gameover_rect)
            
            final_score = font_large.render(f"Natija: {score} ta o'ldirish", True, YELLOW)
            final_rect = final_score.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(final_score, final_rect)
            
            restart_text = font_medium.render("ESC - Menuga qaytish", True, WHITE)
            restart_rect = restart_text.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 80))
            screen.blit(restart_text, restart_rect)
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        game_state = 'menu'
        
        pygame.display.flip()
        clock.tick(FPS)
    
    pygame.quit()


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'pygame'